In [1]:
import tensorflow as tf

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [3]:
def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

In [4]:
class SubBlock(tf.keras.layers.Layer):
    def __init__(self,dim):
        super().__init__()
        self.dim = dim
        self.conv1 = tf.keras.layers.Conv2D(dim//4,1,activation=mish)
        self.conv2 = tf.keras.layers.Conv2D(dim//4,3,activation=mish,padding='same')
        self.conv3 = tf.keras.layers.Conv2D(dim,1)
        
    def build(self,inp_shape):
        if inp_shape[-1] != self.dim:
            self.transfer = tf.keras.layers.Conv2D(self.dim,1)
            self.need_transfer = True
        else:
            self.need_transfer = False
        
    def call(self,inp):
        x = self.conv1(inp)
        x = self.conv2(x)
        x = self.conv3(x)
        if self.need_transfer:
            inp = self.transfer(inp)
        x = x + inp
        x = mish(x)
        return x
    
class Block(tf.keras.layers.Layer):
    def __init__(self,dim,subblocks):
        super().__init__()
        self.subblocks = [SubBlock(dim) for _ in range(subblocks)]
        self.maxpooling = tf.keras.layers.MaxPooling2D(2,2)
    def call(self,x):
        for s in self.subblocks:
            x = s(x)
        x = self.maxpooling(x)
        return x

In [6]:
model = tf.keras.Sequential([
    Block(8,3), #16
    Block(16,3), #8
    Block(32,3), #4
    Block(64,3), #2
    Block(128,3), #1
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64,activation=mish),
    tf.keras.layers.Dense(32,activation=mish),
    tf.keras.layers.Dense(16,activation=mish),
    tf.keras.layers.Dense(10),
])

In [7]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [8]:
model.fit(x_train/255,y_train,batch_size=16,epochs=10,validation_data=(x_test/255, y_test))

Epoch 1/10
3125/3125 [==============================] - 224s 70ms/step - loss: 1.8963 - sparse_categorical_accuracy: 0.3003 - val_loss: 1.7403 - val_sparse_categorical_accuracy: 0.3749
Epoch 2/10
3125/3125 [==============================] - 220s 70ms/step - loss: 1.5316 - sparse_categorical_accuracy: 0.4437 - val_loss: 1.4430 - val_sparse_categorical_accuracy: 0.4767
Epoch 3/10
3125/3125 [==============================] - 222s 71ms/step - loss: 1.3790 - sparse_categorical_accuracy: 0.5044 - val_loss: 1.3089 - val_sparse_categorical_accuracy: 0.5245
Epoch 4/10
3125/3125 [==============================] - 236s 76ms/step - loss: 1.2829 - sparse_categorical_accuracy: 0.5399 - val_loss: 1.3307 - val_sparse_categorical_accuracy: 0.5349
Epoch 5/10
3125/3125 [==============================] - 239s 76ms/step - loss: 1.2114 - sparse_categorical_accuracy: 0.5659 - val_loss: 1.2473 - val_sparse_categorical_accuracy: 0.5554
Epoch 6/10
3125/3125 [==============================] - 238s 76ms/step - lo

3125/3125 [==============================] - 217s 69ms/step - loss: 1.0136 - sparse_categorical_accuracy: 0.6435 - val_loss: 1.1302 - val_sparse_categorical_accuracy: 0.6099racy: 0.654 - ETA: 3:02 - loss: 0.9797 - sparse_categorical_accuracy:  - ETA: 3:01 - loss: 0.9802 - sparse_c - ETA: 2:59 - loss: 0.9824 - sparse_categorical_accuracy: 0. - ETA: 2:59 - loss: 0.9835 - spar - ETA: 2:57 - loss: 0.9815 - sparse_categoric - ETA: 2:56 - loss: 0.9778 - sparse_categorical_acc - ETA: 2:55 - loss: 0.9780 - sparse_categorical_accuracy: 0 - ETA: 2:54 - loss: 0.9777 - sparse_categorical_ - ETA: 2:53 - loss: 0.9799 - - ETA: 2:51 - loss: 0.9830 - sparse_categ - ETA: 2:49 - loss: 0.9827 - sparse_categorical_accuracy: 0.65 - ETA: 2:49 - loss: 0.9838 - sparse_c - ETA: 2:47 - loss: 0.9855 - sparse_catego - ETA: 2:45 - loss: 0.9832 - sparse_categorical_accuracy: 0 - ETA: 2:45 - loss: 0.9838  - ETA: 2:43 - loss: 0.9845 - sparse_categorical_accuracy: 0.651 - ETA: 2:43 - loss: 0.9846 - sparse_categorical_a

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block (Block)                (16, 16, 16, 8)           262       
_________________________________________________________________
block_1 (Block)              (16, 8, 8, 16)            1000      
_________________________________________________________________
block_2 (Block)              (16, 4, 4, 32)            3824      
_________________________________________________________________
block_3 (Block)              (16, 2, 2, 64)            14944     
_________________________________________________________________
block_4 (Block)              (16, 1, 1, 128)           59072     
_________________________________________________________________
flatten (Flatten)            (16, 128)                 0         
_________________________________________________________________
dense (Dense)                (16, 64)                  8